# 03b - Entraînements variantes
Boucle automatique sur modèles/optimiseurs/dropouts/augmentations et jeux 3/4 classes.

In [ ]:
from pathlib import Path
import sys

# Détection racine projet
ROOT = Path.cwd().resolve()
for _ in range(10):
    if (ROOT / "Data").exists():
        break
    ROOT = ROOT.parent
else:
    raise FileNotFoundError("Racine projet introuvable (dossier 'Data' non trouvé).")

# Paths data
TRAIN_DIR = ROOT / "Data" / "raw" / "train"
VAL_DIR   = ROOT / "Data" / "raw" / "val"

# Logs
LOG_DIR = ROOT / "Monitoring" / "output"
LOG_DIR.mkdir(parents=True, exist_ok=True)

# Python : privilégier .venv si dispo
VENV_PY = ROOT / ".venv" / "Scripts" / "python.exe"
PY = str(VENV_PY if VENV_PY.exists() else Path(sys.executable))

print("ROOT =", ROOT)
print("PY   =", PY)
print("TRAIN_DIR =", TRAIN_DIR)
print("VAL_DIR   =", VAL_DIR)
print("LOG_DIR   =", LOG_DIR)


In [ ]:
from subprocess import run

# Grille de variantes (adapter selon ressources)
models = ["resnet18", "resnet34", "baseline"]
optimizers = ["adam", "rmsprop", "adagrad"]
dropouts = [0.2, 0.4]
augment_modes = ["light", "realistic"]
class_sets = [
    ["Chao", "Ervas", "Milho"],
    ["Chao", "Ervas", "Milho", "Milho_ervas"],
]

limit_per_class = 800   # None pour full dataset
epochs = 5
batch_size = 32
image_size = 224

WEIGHTS_DIR = ROOT / "Model" / "weights"
WEIGHTS_DIR.mkdir(parents=True, exist_ok=True)

def run_variant(model: str, optimizer: str, dropout: float, augment: str, classes: list[str]) -> Path:
    suffix = f"{model}_{optimizer}_do{int(dropout*100)}_{augment}_{len(classes)}cls"
    save_path = WEIGHTS_DIR / f"best_{suffix}.pt"

    cmd = [
        PY, "-m", "Model.training.train",  
        "--train-dir", str(TRAIN_DIR),
        "--val-dir", str(VAL_DIR),
        "--model", model,
        "--optimizer", optimizer,
        "--epochs", str(epochs),
        "--batch-size", str(batch_size),
        "--dropout", str(dropout),
        "--image-size", str(image_size),
        "--augment", augment,
        "--save-path", str(save_path),
        "--log-dir", str(LOG_DIR),
    ]

    # Si tu veux forcer le filtre classes en 3-classes uniquement
    if len(classes) == 3:
        cmd += ["--class-filter", *classes]

    if limit_per_class is not None:
        cmd += ["--limit-per-class", str(limit_per_class)]

    print("===", " ".join(cmd))
    run(cmd, check=True, cwd=str(ROOT))   
    return save_path


In [ ]:
results = []

for model in models:
    for optimizer in optimizers:
        for dropout in dropouts:
            for augment in augment_modes:
                for cls_set in class_sets:
                    save_path = run_variant(model, optimizer, dropout, augment, cls_set)
                    results.append({
                        "model": model,
                        "optimizer": optimizer,
                        "dropout": dropout,
                        "augment": augment,
                        "classes": cls_set,
                        "weights": str(save_path),
                    })

print("Variantes terminées :", len(results))
